In [1]:
# Import packages to help with data cleaning, structuring, and aggregation
import pandas as pd

In [3]:
# Import the Mixpanel time to sequence event data

df = pd.read_csv('df.csv')

In [8]:
# Convert the time variable into a datetime variable for easier calculation in the future
df['event_time'] = pd.to_datetime(df['event_time'], format='%Y-%m-%d %H:%M:%S')

In [45]:
# df['button_name'].unique()

In [39]:
# For each route, create 'blocks' of events as the driver can start the sequencing process multiple times
# But we want to count all of these individually and not as 1 block, that will lead to severe overestimation

route_id = []
start_event_button_name = []
start_event_time = []

end_event_button_name = []
end_event_time = []
time_between = []

for i in df['route_id'].unique():
    temp = df.loc[df['route_id']==i].reset_index(drop=True)

    temp2 = temp.loc[temp['button_name']=='start_sequencing'].reset_index()
    if len(temp2) == 0:
        continue

    temp3 = temp.loc[temp['button_name']=='confirm_sequence'].reset_index()
    if len(temp3) == 0:
        continue
    # print(temp2)

    start = []
    end = []

    for j in range(0,len(temp2['index'])-1):
        cur = temp2.iloc[j]
        nex = temp2.iloc[j+1]
        # print(cur)
        # print(nex)
        # print()

        start.append(cur['index'])
        end.append(nex['index'])

    start.append(temp2.iloc[-1]['index'])
    end.append(temp3.iloc[0]['index'])
    order = pd.DataFrame({'start':start,'end':end})

    for x in range(0,len(order)):
        temp4 = order.iloc[x]
        
        temp5 = temp.iloc[temp4['start']:temp4['end']]
        if len(temp5) == 0:
            continue
            
        route_id.append(i)
        start_event_button_name.append(temp5.iloc[0]['button_name'])
        start_event_time.append(temp5.iloc[0]['event_time'])

        end_event_button_name.append(temp5.iloc[-1]['button_name'])
        end_event_time.append(temp5.iloc[-1]['event_time'])
        time_between.append( (temp5.iloc[-1]['event_time']-temp5.iloc[0]['event_time']).seconds/60 )

In [41]:
df2 = pd.DataFrame({'route_id':route_id,'start_event_button_name':start_event_button_name,'start_event_time':start_event_time,
              'end_event_button_name':end_event_button_name,'end_event_time':end_event_time,
              'time_between':time_between})

In [47]:
# Combining all the individual times of each route

df3 = df2.groupby('route_id')['time_between'].sum().reset_index()
df3

,route_id,time_between
0,59112667,7.583333
1,59112670,3.183333
2,59112684,3.033333
3,59112690,2.700000
4,59112713,4.083333
...,...,...
3077,60178590,0.066667
3078,60180818,3.000000
3079,60181567,18.766667
3080,60186118,5.266667
